In [59]:
import os
from pprint import pprint
from proxy import Resolve

r = Resolve('/Volumes/DIT-2T#3/220826-虾米音乐节@阿那亚/素材/220826-Day_01')

r.project.GetSetting()


for subfolder in r.root_folder.GetSubFolderList():
    for clip in subfolder.GetClipList():
        # clip.SetClipProperty('Input Color Space', 'DJI D-Gamut/D-Log')
        clip_path = clip.GetClipProperty('File Path')
        # print(clip_path)
        cam_name = clip_path.split('/')[clip_path.split('/').index(os.path.basename(r.path)) + 1].split('#')[0]
        print(cam_name)

DV
DV
DV
DV
DV
DV
DV
DV
DV
DV
DV
DV
DV
DV
DV
DV
DV
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
GH5M2
G